# Once trained, extract a faster version of the GP forward mean prediction

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import os
from link_bot_gaussian_process import link_bot_gp, data_reformatting
from link_bot_pycommon import link_bot_pycommon, link_bot_sdf_utils
import scipy
from link_bot_data.multi_environment_datasets import MultiEnvironmentDataset
import tensorflow as tf
from IPython.display import HTML
import warnings
from matplotlib import animation
import gpflow as gpf
import h5py

tf.logging.set_verbosity(tf.logging.ERROR)
np.set_printoptions(suppress=True, precision=4, linewidth=200)
plt.style.use("slides")
plt.style.use("slides")

class HackObject:
    pass
    
args = HackObject()
args.mask_label_type = HackObject()
args.mask_label_type.name = "Combined"

Writer = animation.writers['ffmpeg']
writer = Writer(fps=30, metadata=dict(artist='Me'), bitrate=1800)

In [2]:
sdf, _, _, _ = link_bot_sdf_utils.load_sdf("/home/pmitrano/catkin_ws/src/link_bot/link_bot_data/data/sdfs/empty_sdf.npz")
extent = [-5, 5, -5, 5]

In [3]:
gp_model_dir = '/home/pmitrano/catkin_ws/src/link_bot/link_bot_gaussian_process/log_data/separate_independent/July_15_17-01-05__7603552a12__gpf/'

# gpf.reset_default_graph_and_session()
# gpf.reset_default_session()
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=False, per_process_gpu_memory_fraction=0.1))
gpf.reset_default_graph_and_session(config=config)
sess = gpf.get_default_session()

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=h5py.h5py_warnings.H5pyDeprecationWarning)
    fwd_gp_model = link_bot_gp.LinkBotGP()
    fwd_gp_model.load(os.path.join(gp_model_dir, 'fwd_model'))
    
    inv_gp_model = link_bot_gp.LinkBotGP()
    inv_gp_model.load(os.path.join(gp_model_dir, 'inv_model'))

Loading model from /home/pmitrano/catkin_ws/src/link_bot/link_bot_gaussian_process/log_data/separate_independent/July_15_17-01-05__7603552a12__gpf/fwd_model
Loading model from /home/pmitrano/catkin_ws/src/link_bot/link_bot_gaussian_process/log_data/separate_independent/July_15_17-01-05__7603552a12__gpf/inv_model


In [4]:
X = fwd_gp_model.model.X._value
Y = fwd_gp_model.model.X._value
s = np.array([[0.5, 0.5, 0.5, 0, 0, 0]])
u = np.array([[0.2, 0.2]])
s_relative = data_reformatting.make_relative_to_head(s)
Xnew = np.hstack((s_relative, u))

In [5]:
delta_mu, _ = fwd_gp_model.model.predict_y(Xnew)
print(delta_mu)

[[0.0069 0.0056 0.0165 0.0014 0.0177 0.019 ]]


In [30]:
num_inducing = len(fwd_gp_model.model.feature)
mean_X = sess.run(fwd_gp_model.model.mean_function(fwd_gp_model.model.X._value))
err = fwd_gp_model.model.Y._value - mean_X
Kuf = sess.run(fwd_gp_model.model.feature.Kuf(fwd_gp_model.model.kern, fwd_gp_model.model.X._value))
Kuu = sess.run(fwd_gp_model.model.feature.Kuu(fwd_gp_model.model.kern))
sigma = np.sqrt(fwd_gp_model.model.likelihood.variance._value)
L = np.linalg.cholesky(Kuu)
LB = np.ndarray((6, num_inducing, num_inducing))
c = np.ndarray((6, num_inducing))

Kus = sess.run(fwd_gp_model.model.feature.Kuf(fwd_gp_model.model.kern, Xnew))
for i in range(6):
    L_i = L[i]
    Kuf_i = Kuf[i]
    sigma_i = sigma[i]
    err_i = err[:, i]
    A_i = scipy.linalg.solve_triangular(L_i, Kuf_i, lower=True) / sigma_i
    B_i = np.matmul(A_i, A_i.T) + np.eye(num_inducing)
    LB_i = np.linalg.cholesky(B_i)
    LB[i] = LB_i
    Aerr_i = np.matmul(A_i, err_i)
    c_i = scipy.linalg.solve_triangular(LB_i, Aerr_i, lower=True) / sigma_i
    c[i] = c_i

In [33]:
%%timeit
Kus = sess.run(fwd_gp_model.model.feature.Kuf(fwd_gp_model.model.kern, Xnew))
for i in range(6):
    L_i = L[i]
    LB_i = LB[i]
    Kus_i = Kus[i]
    c_i = c[i]
    tmp1_i = scipy.linalg.solve_triangular(L_i, Kus_i, lower=True)
    tmp2_i = scipy.linalg.solve_triangular(LB_i, tmp1_i, lower=True)
    mean[i] = np.matmul(tmp2_i.T, c_i)
# print(mean)

858 ms ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
Knm = fwd_gp_model.model.kern.compute_K(x_star, X_inducing_points)
Kmm = fwd_gp_model.model.kern.compute_K_symm(X_inducing_points)
Knn = fwd_gp_model.model.kern.compute_K_symm(x_star)
Kmn = Knm.T
Kmm_inv = np.linalg.lstsq(Kmm, np.eye(N), rcond=1)[0]
f = Y_inducing_points
f_star_mean = Knm@Kmm_inv@f
print(f_star_mean)
f_star_sigma = Knn - Knm@Kmm_inv@Kmn
print(f_star_sigma)

In [ ]:
initial_config = np.array([[0.5, 0.5, 0.5, 0, 0, 0]])
steps = 150
vx = -0.3
vy = 0.3
controls =  np.tile([vx,vy], [steps, 1])
durations = np.ones(steps, dtype=np.int)
prediction, variances = link_bot_gp.predict(fwd_gp_model, initial_config, controls, durations)
anim = link_bot_gp.animate_predict(prediction, sdf, arena_size=5, linewidth=2)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
# anim.save('new_gp_prediction.mp4', writer=writer)

In [ ]:
plt.figure()
plt.plot(variances[:, 0], label='dx1')
plt.plot(variances[:, 1], label='dy1')
plt.plot(variances[:, 2], label='dx2')
plt.plot(variances[:, 3], label='dy2')
plt.plot(variances[:, 4], label='dx3')
plt.plot(variances[:, 5], label='dy3')
plt.plot(np.linalg.norm(controls * 1e-4, axis=1), color='k')
plt.xlabel("time")
plt.ylabel("variance")
plt.legend();

In [ ]:
inv_gp_model.initialize_rng(1, 100)

In [ ]:
test_dataset = MultiEnvironmentDataset.load_dataset("./data/gz_test_231723146d_50_100/dataset.json")
env = test_dataset.environments[0]
rope_configurations = env.rope_data['rope_configurations']

In [ ]:
print(rope_configurations[0])
print(rope_configurations[10])

In [ ]:
s = np.expand_dims(rope_configurations[0], axis=0)
s_target = np.expand_dims(rope_configurations[24], axis=0)
s = np.array([[2, -1, 2.5, -1, 2.5, -1.5]])
s_target = np.array([[0, 0, 0.5, 0, .5, 0.5]])
s = np.expand_dims(link_bot_pycommon.make_random_rope_configuration(extent), axis=0)
s_target = np.expand_dims(link_bot_pycommon.make_random_rope_configuration(extent), axis=0)
u, _ = inv_gp_model.inv_act(s, s_target)
plt.plot([s[0, 0], s[0, 2], s[0, 4]], [s[0,1], s[0,3], s[0,5]], c='r')
plt.plot([s_target[0, 0], s_target[0, 2], s_target[0, 4]], [s_target[0,1], s_target[0,3], s_target[0,5]])
plt.scatter([s[0, 4], s_target[0,4]], [s[0,5], s_target[0,5]], c='k')
plt.quiver(s[0,4], s[0,5], u[0,0], u[0,1])
plt.axis("equal");

In [ ]:
156038/14380

In [ ]:
21972/14401